<a href="https://colab.research.google.com/github/dotsnangles/Retrieval-Based-Chatbot/blob/main/chatbot_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !rm -rf /content/Poly-Encoder

In [1]:
!nvidia-smi

Thu Jul  7 00:21:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!gdown -q --folder 1Ipr-aNF5ELMY0HTXAmeV26LlgktKUfmG

In [4]:
!git clone https://github.com/dotsnangles/Poly-Encoder.git

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 129 (delta 41), reused 20 (delta 8), pack-reused 57
Receiving objects: 100% (129/129), 40.70 KiB | 13.57 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [5]:
!pip install -q transformers datasets folium==0.2.1

     |████████████████████████████████| 4.4 MB 8.0 MB/s 
     |████████████████████████████████| 362 kB 99.1 MB/s 
     |████████████████████████████████| 69 kB 9.3 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 6.6 MB 80.1 MB/s 
     |████████████████████████████████| 140 kB 95.0 MB/s 
     |████████████████████████████████| 1.1 MB 70.3 MB/s 
     |████████████████████████████████| 212 kB 100.9 MB/s 
     |████████████████████████████████| 127 kB 84.4 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 101.4 MB/s 
     |████████████████████████████████| 144 kB 69.6 MB/s 


In [6]:
%cd /content/Poly-Encoder

/content/Poly-Encoder


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertConfig, BertModel, BertTokenizer, AutoModel
from encoder import PolyEncoder
from transform import SelectionJoinTransform, SelectionSequentialTransform

PATH = '/content/Poly-Encoder/chatbot_output/poly_16_pytorch_model.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens=True)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=256)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=128)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
# model.load_state_dict(torch.load(PATH))

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(32001, 768)

In [8]:
context = ['This framework generates embeddings for each input sentence', 
           'Sentences are passed as a list of string.', 
           'The quick brown fox jumps over the lazy dog.']

candidates = ['This framework generates embeddings for each input sentence']

In [9]:
context_input_ids, context_input_masks = context_transform(context)
responses_token_ids_list, responses_input_masks_list = response_transform(candidates)

In [10]:
contexts_token_ids_list_batch, contexts_input_masks_list_batch, \
responses_token_ids_list_batch, responses_input_masks_list_batch = [context_input_ids], [context_input_masks], [responses_token_ids_list], [responses_input_masks_list]

long_tensors = [contexts_token_ids_list_batch, contexts_input_masks_list_batch,
                                            responses_token_ids_list_batch, responses_input_masks_list_batch]

contexts_token_ids_list_batch, contexts_input_masks_list_batch, \
responses_token_ids_list_batch, responses_input_masks_list_batch = (torch.tensor(t, dtype=torch.long) for t in long_tensors)

In [11]:
def embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
        
        ctx_out = model.bert(contexts_token_ids_list_batch, contexts_input_masks_list_batch)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(model.poly_m, dtype=torch.long).to(contexts_token_ids_list_batch.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(1, model.poly_m)
        poly_codes = model.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = model.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        return embs

In [12]:
embs = embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch)

In [13]:
def cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch):

    with torch.no_grad():
        model.eval()
                
        batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape # res_cnt is 1 during training
        responses_token_ids_list_batch = responses_token_ids_list_batch.view(-1, seq_length)
        responses_input_masks_list_batch = responses_input_masks_list_batch.view(-1, seq_length)
        cand_emb = model.bert(responses_token_ids_list_batch, responses_input_masks_list_batch)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        return cand_emb

In [14]:
cand_emb = cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch)

In [15]:
def score(embs, cand_emb):
    with torch.no_grad():
        model.eval()

        ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
        dot_product = (ctx_emb*cand_emb).sum(-1)
        
        return dot_product

In [16]:
score(embs, cand_emb)

tensor([[275.2350]])

In [17]:
# forward
with torch.no_grad():
    model.eval()
    
    model_foward = model(contexts_token_ids_list_batch, contexts_input_masks_list_batch, responses_token_ids_list_batch, responses_input_masks_list_batch)
    print(model_foward)

tensor([[275.2350]])


### 데이터 검증

In [18]:
import pickle

with open('/content/감성대화챗봇데이터/train_data_source.pickle', 'rb') as f:
    train = pickle.load(f)
with open('/content/감성대화챗봇데이터/val_data_source.pickle', 'rb') as f:
    dev = pickle.load(f)

In [39]:
index = 500

In [40]:
train[index]['context']

['participant 1: 이번에 상사가 큰 실수를 저질렀어. 너무 당황스러워.',
 'participant 2: 상사의 실수 때문에 당황스러우시군요.',
 'participant 1: 매번 팀원들에게 일을 떠넘기기만 하더니 결국 일을 저질렀네.',
 'participant 2: 상사가 일을 떠넘기다가 실수를 했다고 생각하시는군요. 당황한 마음이 조금이라도 진정되려면 어떻게 해야 좋을까요?',
 'participant 1: 나는 조용히 있어야지. 나섰다가 더 큰일이 나면 어떡해.']

In [41]:
train[index]['responses']

['큰일이 나지 않게 가만히 있으려고 하시는군요.',
 '흰옷을 입고 결혼식장에 갈까 하는 생각까지도 드셨나 보네요.',
 '요양원 사람들과 잘 어울려서 부정적인 감정은 잊어버리실 수 있길 바라요.',
 '언니와의 연락을 통해 언니의 걱정하는 마음이 조금이나마 덜어지길 바랄게요.',
 '피해다니시는 것이 상책이라 생각하시는 군요.',
 '어떤 걸 먹을 생각이신가요?',
 '초심으로 돌아가기로 하셨군요. 일에 전념하여 지금의 기분이 나아지셨으면 좋겠어요.',
 '아들과 대화를 통해 결정을 내리려 하시는군요.',
 '자신의 심정을 부모님께 이야기할까 고려 중이군요.',
 '아내에 대한 사랑으로 잘 극복해 나가길 바라요.',
 '그렇군요. 오늘 좋은 외국인 친구를 만들기 바라요.',
 '주변의 도움으로 좋은 방법을 얻을 수 있길 바라요.',
 '체력이 증진되어 꾸준히 운동한 보람을 느끼시는군요.',
 '힘든 상황을 정면으로 돌파하실 생각이시군요.',
 '야근하지 않아도 되게끔 업무가 원만히 해결되길 바랍니다.',
 '성적으로 차별하시는 어머니께 혐오스러운 마음이 드시는 군요.']

In [42]:
dev[index]['context']

['participant 1: 이번 시험은 쉬워서 마음이 편해.',
 'participant 2: 걱정이 적어지셨겠어요. 이제 무엇을 하실 건가요?',
 'participant 1: 내일도 시험이니까 열심히 공부할래.']

In [43]:
dev[index]['responses']

['좋아요. 내일 시험도 좋은 결과가 있기를 바랄게요.',
 '엄마와의 대화에서 힌트를 얻었으면 좋겠네요.',
 '당분간 휴식을 취하면서 원하는 바를 이루시길 바라요.',
 '가족 관계 문제로 많이 불안하시겠네요.',
 '아내와의 접촉을 줄여서 관계가 회복되길 바라요.',
 '경제적인 문제가 많이 나아졌으면 좋겠어요.',
 '동생에게도 영어를 가르쳐주려고 하시는군요.',
 '주변 지인들에게 도움을 청하려고 하시는군요. 원하시는대로 상황이 나아지길 바랄게요.',
 '친구 분들과의 대화로 영양제 문제가 해결되었으면 좋겠네요.',
 '마음에 드는 좋은 아르바이트를 찾았으면 좋겠어요.',
 '면역력을 높일 수 있는 운동을 많이 하시는군요.',
 '준비하신 만큼 좋은 결과가 있기를 바라요.',
 '직업상담을 받으시는군요. 도움이 되길 바라요.',
 '형편이 안 좋아서 병원에 못 가는 일은 없었으면 좋겠다고 생각하시는군요.',
 '가족에게 다이어트 얘기를 들어서 기분이 언짢으시군요.',
 '더 좋은 기회를 위한 발판으로 생각해보시는건 어떠세요?',
 '사춘기가 지나면 다시 모녀지간이 가까워지길 소망합니다.',
 '좋아하는 팝송을 들으며 영어 단어 학습이 잘 되길 바랄게요.',
 '좌절감을 주는 대인관계와 거기서 오는 실망감으로 괴로우시군요.',
 '건강하시길 바라요.',
 '목표가 있고 배려심이 깊은 사람이 이상형이시군요.',
 '스스로의 선택을 후회하지 않으시는 것 같아요.',
 '직장상사와 친해지시면 좋겠네요.',
 '고양이는 사랑스러운 동물이죠.',
 '네. 더욱더 건강해지시길 기원합니다.',
 '도시락을 맘대로 뺏어 먹지 않도록 단호하게 말하려 하시는군요.',
 '건강검진을 받으려 하시는군요.',
 '건강 문제로 걱정이 많으시군요.',
 '좋은 방법이에요. 잘 극복하길 바라요.',
 '친한 친구와의 진솔한 대화로 상황이 개선되길 바랄게요.',
 '그렇군요. 너무 안타까워요.',
 '주변 사람들과 대화로 상황이 나아졌으면 좋겠어요.',
 '기억이 나지 않아서 많이 